In [16]:
#Data Pre-precessing

import numpy as np
import pandas as pd
import csv
from sklearn import preprocessing

def word2num(x,y):
    return [i for i in range(len(y)) if y[i] == x]

def data_precessing(source_datafile,handled_datafile):
    file_data = open(handled_datafile,'w',newline='') #Write dataset

    with open(source_datafile,'r') as data_soure:
        csv_read = csv.reader(data_soure)
        csv_write = csv.writer(file_data)
        for row in csv_read:
            temp = np.array(row)
            
            #Protocol
            protocol_list = ['tcp','udp','icmp']
            if temp[1] in protocol_list:
                temp[1] = word2num(temp[1],protocol_list)[0]

            #Service
            service_list = ['aol','auth','bgp','courier','csnet_ns','ctf','daytime','discard','domain','domain_u',
                    'echo','eco_i','ecr_i','efs','exec','finger','ftp','ftp_data','gopher','harvest','hostnames',
                    'http','http_2784','http_443','http_8001','imap4','IRC','iso_tsap','klogin','kshell','ldap',
                    'link','login','mtp','name','netbios_dgm','netbios_ns','netbios_ssn','netstat','nnsp','nntp',
                    'ntp_u','other','pm_dump','pop_2','pop_3','printer','private','red_i','remote_job','rje','shell',
                    'smtp','sql_net','ssh','sunrpc','supdup','systat','telnet','tftp_u','tim_i','time','urh_i','urp_i',
                    'uucp','uucp_path','vmnet','whois','X11','Z39_50']
            if temp[2] in service_list:
                temp[2] = word2num(temp[2],service_list)[0]
        
            #Flag
            flag_list = ['OTH','REJ','RSTO','RSTOS0','RSTR','S0','S1','S2','S3','SF','SH']
            if temp[3] in flag_list:
                temp[3] = word2num(temp[3],flag_list)[0]

            #State
            if(temp[41] == 'normal.'):
                temp[41] = 1
            else:
                temp[41] = 0

            csv_write.writerow(temp)
        file_data.close()
    
    #One-Hot Encoding
    with open(handled_datafile) as onehot_file:
        data_matrix = np.loadtxt(onehot_file, delimiter=",")

    enc = preprocessing.OneHotEncoder()
    enc.fit(data_matrix[:,[1,2,3]])
    en_matrix = np.zeros((len(data_matrix),80))

    for i in range(len(data_matrix)):
        en_temp = enc.transform([data_matrix[i,[1,2,3]]]).toarray()
        en_matrix[i,:] = en_temp
    
    np.savetxt('/Users/ziyuanjiang/dstoolbox/datasetkdd/onehotdata.csv',en_matrix.astype(int),delimiter=',',fmt='%.0f')
    
    data_fin = np.hstack((en_matrix,np.delete(data_matrix,[1,2,3],axis=1)))
    np.savetxt('/Users/ziyuanjiang/dstoolbox/datasetkdd/onehotdata1.csv',data_fin.astype(int),delimiter=',',fmt='%.0f')
    
    return 0


In [17]:
source_datafile = '/Users/ziyuanjiang/dstoolbox/datasetkdd/testdata'
handled_datafile = '/Users/ziyuanjiang/dstoolbox/datasetkdd/testdata_new.csv'

data_precessing(source_datafile,handled_datafile)

0